In [3]:
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import imageio
from tqdm.notebook import tqdm

In [4]:
# ----- Day 9: Rope Bridge -----

with open("./data_inputs/day09_input.txt") as f:
    input_raw = f.read()

motions = [tuple(motion.split(" ")) for motion in input_raw.split("\n")]

In [5]:
# ---- Part 1 ----

h = [0, 0]  # (x, y)
t = [0, 0]
t_track_log = []

get_rel_pos = lambda h, t: (h[0] - t[0], h[1] - t[1])

for (dir, steps) in motions:
    for _ in range(int(steps)):
        match dir:
            case "U": h[1] += 1      
            case "R": h[0] += 1           
            case "D": h[1] -= 1      
            case "L": h[0] -= 1
            case _: raise ValueError("Wrong direction!")

        rel_pos = get_rel_pos(h, t)

        if rel_pos not in product((-1, 0, 1), repeat=2):
            if rel_pos[0] > 0: t[0] += 1 
            elif rel_pos[0] < 0: t[0] -= 1

            if rel_pos[1] > 0: t[1] += 1
            elif rel_pos[1] < 0: t[1] -= 1 

        t_track_log.append(tuple(t))


pos_visited = len(set(t_track_log))

print("Result 1:", pos_visited)

Result 1: 6209


In [10]:
# ---- Part 2 ----

def create_frame(arr, i, len_t_pos):
    fig = plt.figure(figsize=(5,5))
    plt.imshow(arr)
    plt.title(f"Frame: {i}/2000 - Unique tail positions: {len_t_pos}")
    plt.tight_layout()
    plt.savefig(f"./d09_gif/img_{i}.png", transparent=False)
    plt.close()

rope = {n: [20, 20] for n in range(10)}

t_track_log2 = []
frames = []
i = 0

for (dir, steps) in tqdm(motions):
    for _ in range(int(steps)):
        match dir:
            case "U": rope[0][1] += 1      
            case "R": rope[0][0] += 1           
            case "D": rope[0][1] -= 1      
            case "L": rope[0][0] -= 1
            case _: raise ValueError("Wrong direction!")

        frame = np.zeros((340, 340))

        for knot in range(1, 10):
            rel_pos = get_rel_pos(rope[knot-1], rope[knot])

            if rel_pos not in product((-1, 0, 1), repeat=2):
                if rel_pos[0] > 0: rope[knot][0] += 1 
                elif rel_pos[0] < 0: rope[knot][0] -= 1

                if rel_pos[1] > 0: rope[knot][1] += 1
                elif rel_pos[1] < 0: rope[knot][1] -= 1 

            frame[*rope[knot]] = 20-knot

        if tuple(rope[9]) not in t_track_log2:
            t_track_log2.append(tuple(rope[9]))
    
    if i % 2 == 0:                  # creating only half of the frames/imgs
        for t_pos in t_track_log2:
            frame[*t_pos] = 5
        frame[-1, i%340:(i+5)%340] = 10
        create_frame(frame, i, len(t_track_log2))

    i += 1

for i in range(len(motions)):
    image = imageio.v2.imread(f"./d09_gif/img_{i}.png")
    frames.append(image)

imageio.mimsave("./day09_part_2.gif", frames, fps=25)
pos_visited2 = len(set(t_track_log2))

print("Result 2:", pos_visited2)

  0%|          | 0/2000 [00:00<?, ?it/s]

Result 2: 2460


In [12]:
# OPTIONAL STEP
# make sure that you have gifsicle installed in your OS and in the path
from pygifsicle import optimize

optimize("./day09_part_2.gif")

![SegmentLocal](day09_part_2.gif "segment")